In [1]:

!pip install pandastable

  Created wheel for pandastable: filename=pandastable-0.12.2.post1-py3-none-any.whl size=255038 sha256=277eab5ae5581aaf60e5b4d3ace48cb8e70ffdb9aa49fe9c8d4e50d8611ba969
  Stored in directory: c:\users\mkell\appdata\local\pip\cache\wheels\f7\b3\d1\a095c64d840409f1846e117b2c71b962449e76c670f96bb13e
Successfully built pandastable


In [ ]:
#TODO
data frame changing in window
work out simulation
comment when done
set up location for water and other stuff
get rid of global, use return



In [1]:
from opentrons import protocol_api
import tkinter as tk #basic python package apparently
import csv
import pandas as pd
import numpy as np
import os
from pandastable import Table, TableModel

In [2]:
## Meta Data

metadata = {'protocolName': 'RED',
'author':'Mason Kellinger masonkellinger@vt.edu',
 'description':'Restriction Enzyme Digestion',
 'apiLevel': '2.9'
}
print(metadata)

os.getcwd()

{'protocolName': 'RED', 'author': 'Mason Kellinger masonkellinger@vt.edu', 'description': 'Restriction Enzyme Digestion', 'apiLevel': '2.9'}


'C:\\Users\\mkell\\Documents\\GitHub\\opentrons\\Restriction'

In [3]:

#path = "/Shared drives/PlantSynBioLab/Opentrons/Restriction Enzyme Digestion/csvfiles"
path = "C:/Users/mkell/Desktop/RestrictionTestCSV2.csv"

#config_file = pd.read_csv(path, engine = 'python', encoding='utf-8-sig') ## last 2 parameters are error corrections
config_file = pd.read_csv("CommonEnzyme.csv",engine = 'python', encoding='utf-8-sig')

config_df = pd.DataFrame(config_file)


config_df

,enzyme,incu_temp,incu_time,inac_temp,amount_dna,conc_dna
0,UNITS:,celsius,minutes,celsius,µg,µg/µL
1,SpeI,37,60,80,1,1
2,BbvcI,37,60,NaN,1,1
3,PmeI,37,60,65,1,1
4,SacI,37,60,65,1,1
5,NheI,37,60,65,1,1
6,BsaI,37,60,65,1,1
7,NotI,37,60,65,1,1


In [8]:
from tkinter import *

    # setup window
root = Tk()

    # window size 
root.geometry("500x500")
root.title("Enzyme Entry")

    # make text entry box 
entry = Entry(root)
entry.pack()

label3 = Label(root,text = config_df["enzyme"])  #lists possible enzymes
label3.place(relx = 0.2, rely = 0.2, anchor = "center")

def enter():
    label1 = Label(root,text = entry.get()) # shows what you typed in. not too useful
    label1.pack()
   
    # assign global variable to use outside of `enter` function 
    global enzyme_name
    enzyme_name = "SpeI"  # this is if the user decides to not enter anything
    enzyme_name = entry.get()
  
    # display entered value and its values in the dataframe 
    label2 = Label(root,text =  str(config_file.loc[config_file.enzyme==enzyme_name]))
    label2.place(relx = 0.5,rely = 0.5,anchor = 'center')
    return enzyme_name
    


def confirm():
    root.destroy()

button1 = Button(root,text="Enter your enzyme",command = enter)
button1.pack()

button2 = Button(root,text="Confirm your data, if incorrect fix in table",command = confirm)
button2.place(relx = 0.5,rely = 0.7,anchor = 'center')

    # startup window loop  
root.mainloop()

enzyme_name

'SpeI'

In [5]:
# maybe you could input the index of your enzyme as n and then iloc[n,1] ect.


incubation_temp = int(config_file.loc[config_file.enzyme==enzyme_name,"incu_temp"])
incubation_time = int(config_file.loc[config_file.enzyme==enzyme_name,"incu_time"])
inactivation_temp = int(config_file.loc[config_file.enzyme==enzyme_name,"inac_temp"])
amount_of_dna = int(config_file.loc[config_file.enzyme==enzyme_name,"amount_dna"])
concentration_of_dna = int(config_file.loc[config_file.enzyme==enzyme_name,"conc_dna"])

volume_dna = amount_of_dna * (1/concentration_of_dna)
volume_water = 17 - volume_dna



NameError: name 'enzyme_name' is not defined

In [6]:
from tkinter import *
from pandastable import Table, TableModel

class TestApp(Frame):
        """Basic test frame for the table"""
        def __init__(self, parent=None):
            self.parent = parent
            Frame.__init__(self)
            self.main = self.master
            self.main.geometry('600x400+200+100')
            self.main.title('Table app')
            f = Frame(self.main)
            f.pack(fill=BOTH,expand=1)
            df = config_df
            self.table = pt = Table(f, dataframe=df,
                                    showtoolbar=True, showstatusbar=True)
            pt.show()
            return

app = TestApp()
#launch the app
app.mainloop()


In [7]:
import tkinter as tk
import pandas as pd
from tkinter import filedialog, messagebox, ttk

root = Tk()
root.geometry("500x500")
root.pack_propagate(False)
root.resizable(0,0)

#Frame for Table
frame1 = LabelFrame(root,text = "Configuration Data")
frame1.place(height=250,width=500)

#Frame for file manipulation

file_frame = LabelFrame(root,text = "Open File")
file_frame.place(height=100,width =400,rely=0.65,relx=0)

#Buttons for file searching
button3 = Button(file_frame, text ="Search for File")
button3.place(rely=0.65,relx=0.50)

button4 = Button(file_frame,text = "Confirm File")
button4.place(rely=0.65,relx=0.30)

label_file = Label(file_frame,text= "No File Selected")
label_file.place(rely=0,relx=0)

#Visualing Widget
tv1 = ttk.Treeview(frame1)
tv1.place(relheight=1,relwidth=1)

treescrolly = Scrollbar(frame1,orient="vertical",command = tv1.yview)
treescrollx = Scrollbar(frame1,orient="horizontal",command = tv1.xview)
tv1.configure(xscrollcommand=treescrollx.set,yscrollcommand=treescrolly.set)
treescrollx.pack(side="bottom",fill="x")
treescrolly.pack(side="right",fill="y")




root.mainloop()

In [28]:
# Protocol 

from opentrons import simulate
protocol = simulate.get_protocol_api('2.9')

    ## Loading labware
TempUnit = protocol.load_module('Temperature Module',3)
well = TempUnit.load_labware('opentrons_24_aluminumblock_nest_2ml_snapcap')
    
tiprack1 = protocol.load_labware("opentrons_96_tiprack_10ul",'1')
tiprack2 = protocol.load_labware("opentrons_96_tiprack_20ul","2")
tuberack = protocol.load_labware("opentrons_24_tuberack_nest_2ml_snapcap","4")
    
        ## Loading tools
pipette_left = protocol.load_instrument('p20_single_gen2','left',tip_racks = [tiprack1])
pipette_right = protocol.load_instrument('p20_single_gen2','right',tip_racks = [tiprack2])
    
        ## Operations

    ## get tip -> get x amount of water to fill to 20uL -> dispense into tube
pipette_left.pick_up_tip(tiprack1['A1'])
pipette_left.aspirate(volume_water,location,2)
pipette_left.dispense(volume_water,well['A1'],2) #create function to deteremine amount needed
pipette_left.blow_out()
pipette_left.drop_tip()

    ## get 2uL of 10x buffer -> dispense into tube -> get rid of tip

pipette_left.pick_up_tip()
pipette_left.aspirate(2,location,2)
pipette_left.dispense(2,well['A1'],2)
pipette_left.blow_out()
pipette_left.drop_tip()


    ## get tip -> get y amount of DNA -> dispense into tube -> get rid of tip

pipette_left.pick_up_tip()
pipette_left.aspirate(volume_dna,location,2) #create function to determine
pipette_left.dispense(volume_dna,well['A1'],2)
pipette_left.drop_tip()
    ## we don't blow out so we dont aerosolize dna

    ## get tip -> get 1uL Enzyme -> dispense into tube -> get rid of tip

pipette_left.pick_up_tip()
pipette_left.aspirate(1,location,rate)
pipette_left.dispense(1,well['A1'],1)
pipette_left.blow_out()
pipette_left.drop_tip()
    
    #mix

pipette_left.mix(3,20,well['A1'])  #repetetions, max volume uL, location
    #this helps rinse the extra enzyme as well

## Temp Unit - incubate at temp for time t and then inactivate at temp for 10 min then rest at 4   degrees
                                        
TempUnit.set_temperature(incubation_temp)
protocol.delay(incubation_time)
TempUnit.set_temperature(inactivation_temp)
protocol.delay(10)
TempUnit.set_temperature(4)
    

C:\Users\mkell\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\mkell\.opentrons\deck_calibration.json not found. Loading defaults


NameError: name 'location' is not defined